In [12]:
import json
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import RectBivariateSpline

In [13]:
with open('data/Tracking_example_data_1.json', 'r') as file:
    data = json.load(file)

Layout of the data in the sample boat tracking information

In [14]:
print(data['schema'])
print(data['dframes'][0])

['_lat', '_lon', 'cog', 'sog', 'heading', 'pitch', 'roll', 'wspeed', 'wangle', 'twspeed', 'twangle', 'twdirection', 'vmg']
[-41.285089169939866, 174.79663346037051, 72.2, 5.6, 69.9, 2.6, 2.9, 22.08, -67.2, 18.57, -86.2, 345.4, 0.37]


For a very basic performance analysis twspeed, twangle, vmg are required

In [15]:
minimum_properties = ['twspeed', 'twangle', 'vmg']
indices = [data['schema'].index(property) for property in minimum_properties]
filtered_dframes = [[row[i] for i in indices] for row in data['dframes']]
basic_analysis_data = {
    "schema": minimum_properties,
    "dframes": filtered_dframes
}

In [16]:
print(basic_analysis_data['schema'])
print(basic_analysis_data['dframes'][0])

['twspeed', 'twangle', 'vmg']
[18.57, -86.2, 0.37]


Now, add the specific boat type ORC certified polars (a csv file from https://jieter.github.io/orc-data/site/index.html#AUS/7183)

In [17]:
polars_df = pd.read_csv('data/J112E_polars.csv', delimiter=';')
print(polars_df)

    twa/tws     6     8    10    12    14    16     20
0       0.0  0.00  0.00  0.00  0.00  0.00  0.00   0.00
1      43.0  4.94  0.00  0.00  0.00  0.00  0.00   0.00
2      40.6  0.00  5.82  0.00  0.00  0.00  0.00   0.00
3      39.0  0.00  0.00  6.41  0.00  0.00  0.00   0.00
4      37.8  0.00  0.00  0.00  6.64  0.00  0.00   0.00
5      37.6  0.00  0.00  0.00  0.00  6.78  0.00   0.00
6      37.6  0.00  0.00  0.00  0.00  0.00  6.87   0.00
7      37.8  0.00  0.00  0.00  0.00  0.00  0.00   6.92
8      52.0  5.52  6.52  7.11  7.39  7.51  7.58   7.65
9      60.0  5.81  6.76  7.28  7.56  7.72  7.80   7.89
10     75.0  6.00  6.92  7.42  7.74  7.97  8.14   8.32
11     90.0  6.04  7.08  7.50  7.76  8.06  8.33   8.74
12    110.0  6.17  7.26  7.81  8.19  8.46  8.71   9.17
13    120.0  6.04  7.18  7.78  8.27  8.71  9.10   9.77
14    135.0  5.42  6.72  7.50  8.03  8.55  9.11  10.60
15    150.0  4.60  5.80  6.79  7.45  7.84  8.20   8.99
16    143.9  4.93  0.00  0.00  0.00  0.00  0.00   0.00
17    147.

Interpolate or smooth the polar values. It seems like bilinear interpolation (2D grid) would be most suited in this instance as we want to interpolate both across TWA and TWS at the same time.

Note. Notice the 0 values in the polars. These should be accounted for before performing interpolation as they should be excluded from the interpolation

In [18]:
polars_df.replace(0, np.nan, inplace=True)
polars_df.dropna(how='all', inplace=True)

Now it is possible to setup this data for the interpolation

Setup the 2D grid

In [19]:
# Set up TWA and TWS for the grid and extract boat speed data
twa = polars_df.columns[1:].astype(float).values
tws = polars_df.iloc[1:, 0].astype(float).values
boat_speeds = polars_df.iloc[1:, 1:].values.astype(float)
boat_speeds[np.isnan(boat_speeds)] = np.nanmean(boat_speeds)

# Ensure TWA and TWS are strictly increasing
twa, unique_twa_indices = np.unique(twa, return_index=True)
tws, unique_tws_indices = np.unique(tws, return_index=True)
boat_speeds = boat_speeds[np.ix_(unique_tws_indices, unique_twa_indices)]

In [20]:
# Set up RectBivariateSpline for interpolation
spline = RectBivariateSpline(tws, twa, boat_speeds)
# Interpolate for each TWS and TWA pair in basic analysis data
basic_tws = [row[0] for row in basic_analysis_data['dframes']]
basic_twa = [row[1] for row in basic_analysis_data['dframes']]
interpolated_speeds = spline.ev(basic_tws, basic_twa)

In [21]:
# Add the interpolated speeds to the basic analysis data
for i, row in enumerate(basic_analysis_data['dframes']):
    row.append(interpolated_speeds[i])
basic_analysis_data['schema'].append('interpolated_speed')

# Create a fine grid for a full interpolated speed display
twa_fine = np.linspace(np.min(twa), np.max(twa), 50)
tws_fine = np.linspace(np.min(tws), np.max(tws), 50)
z_fine = spline(tws_fine, twa_fine)

In [22]:
# Create a DataFrame for the interpolated speed grid
df_grid = pd.DataFrame(z_fine, index=tws_fine, columns=twa_fine)
# Print the DataFrame with rounded values
print("Interpolated Speed Grid (rows: TWS, columns: TWA)")
print(df_grid.round(2))

Interpolated Speed Grid (rows: TWS, columns: TWA)
       6.000000   6.285714   6.571429   6.857143   7.142857   7.428571   \
37.6        7.42       7.44       7.45       7.45       7.45       7.44   
40.4        7.43       6.92       6.53       6.26       6.09       6.00   
43.2        7.17       5.24       3.93       3.16       2.86       2.94   
46.0        6.62       4.81       3.60       2.92       2.69       2.85   
48.8        6.01       5.11       4.53       4.24       4.19       4.35   
51.6        5.56       5.63       5.72       5.83       5.95       6.09   
54.4        5.44       5.93       6.31       6.59       6.79       6.91   
57.2        5.59       6.01       6.33       6.58       6.76       6.88   
60.0        5.81       5.98       6.14       6.28       6.42       6.54   
62.8        5.96       5.97       6.02       6.10       6.20       6.33   
65.6        6.04       6.00       6.01       6.06       6.16       6.28   
68.4        6.06       6.04       6.07       6.14 